In [40]:

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from itertools import product
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.ensemble import (RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,BaggingClassifier,VotingClassifier,AdaBoostClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.externals.six import StringIO
#from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
#from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,roc_auc_score,
                             f1_score,accuracy_score,confusion_matrix,mean_absolute_error,r2_score,mean_squared_error)
import datetime

from datetime import datetime

import time
import os
import sys

from scipy import misc
from scipy.stats import randint


In [41]:
now = datetime.today()
today_str= now.strftime("%Y%m%d")

In [2]:
df=pd.read_csv('./data/train.csv')

In [7]:
df.describe()
print("----------------------------example----------------------------")
print(df.head())
print("----------------------------columns----------------------------")
print(df.columns)
print("----------------------------shape----------------------------")
print(df.shape)

----------------------------example----------------------------
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123   

In [55]:
train=df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

def transform_data(df):
    dummy_pclass=pd.get_dummies(df["Pclass"])
    dummy_Sex=pd.get_dummies(df["Sex"])
    dummy_Embarked=pd.get_dummies(df["Embarked"])
    dummy_SibSp=pd.get_dummies(df["SibSp"])
    dummy_Parch=pd.get_dummies(df["Parch"])
    train2=df.drop(['Pclass','Embarked','SibSp','Parch','Sex'],axis=1)
    train2=pd.concat([train2,dummy_pclass],axis=1)
    train2=pd.concat([train2,dummy_Embarked],axis=1)
    train2=pd.concat([train2,dummy_Sex],axis=1)
    train2=pd.concat([train2,dummy_SibSp],axis=1)
    train2=pd.concat([train2,dummy_Parch],axis=1)
    train2=train2.fillna(0)
    return train2
train2=transform_data(train)

In [56]:
train_labels=train2["Survived"]
train_features=train2.drop('Survived',axis=1)

In [33]:
def get_score(clf, train_features, train_labels):
    X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.4, random_state=0)
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))

def get_accuracy(clf, train_features, train_labels):
    scores = cross_validation.cross_val_score(clf, train_features, train_labels, cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

def random_searchSVM(train_features, train_labels, n_jobs,objective):
    param_dist = {
#       {'C': [1, 10, 100], 'kernel': ['linear']},
       #'C': [1,10,50,100], 'gamma': [0.0001, 0.001], 'kernel': ['rbf','linear']}
        'C': [1], 'gamma': [0.001], 'kernel': ['rbf']}

    clf = GridSearchCV(svm.SVC(class_weight="balanced"), param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchGB(train_features, train_labels, n_jobs,objective):
    param_dist = {#'n_estimators': randint(5, 300),
                   #'min_samples_leaf': randint(5, 30),
                   #'max_depth': randint(5, 10),
                   'learning_rate': [0.05,0.001]
                  }

    clf = RandomizedSearchCV(GradientBoostingClassifier(),param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchRF(train_features, train_labels, n_jobs,objective):
    param_dist = {
        #'n_estimators'      : randint(5, 300),
        #'max_features'      : randint(3, 20),
        #'min_samples_split' : randint(3, 100),
        'max_depth'         : randint(3, 100)}

    clf = RandomizedSearchCV(RandomForestClassifier(class_weight="balanced"), param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchVC(train_features,train_labels,n_jobs,objective):
    clf1 = LogisticRegression(random_state=1,class_weight="balanced")
    clf2 = RandomForestClassifier(random_state=1,class_weight="balanced")
    clf3 = GaussianNB()
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
    params = {'lr__C': randint(1, 300), 'rf__n_estimators': randint(5, 300),'rf__max_features':randint(3, 20),'rf__min_samples_split' : randint(3, 100),'rf__max_depth':randint(3, 100)}
    clf = RandomizedSearchCV(estimator=eclf,param_distributions=params,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchAB(train_features,train_labels,n_jobs,objective):
    param_dist = {
        'algorithm'      : ['SAMME', 'SAMME.R'],
        #'n_estimators'      : randint(5, 300)
        }
    bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1))
    clf = RandomizedSearchCV(estimator=bdt,param_distributions=param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_ 

def random_searchNC(train_features,train_labels,n_jobs,objective):
    param_dist = {
        #'activation'      : ['identity', 'logistic', 'tanh', 'relu'],
        #'hidden_layer_sizes' : [(5,2), (10,5),(10,20,5),(5,10,10,5)],
        'alpha' : [1e-5,1e-4,1e-3],
        'solver' : ['adam']}
        #'solver' : ['lbfgs', 'sgd', 'adam']}
      
    clf = GridSearchCV(estimator= MLPClassifier(),param_grid=param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_  

def best_estimater(train_features,train_labels,test_features,test_labels,objective):
    start_time = time.time()
    n_jobs = 30

    #objective = 'f1' or 'accuracy' or 'recall' or 'roc_auc'
    a_RF,b_RF,c_RF=random_searchRF(train_features, train_labels,n_jobs,objective)
  
    print("---RF finished %s seconds ---" % round(time.time() - start_time,2))
    duration_time = time.time()
  
    a_GB,b_GB,c_GB=random_searchGB(train_features, train_labels,n_jobs,objective)
  
    print("---GB finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_SVM,b_SVM,c_SVM=random_searchSVM(train_features, train_labels,n_jobs,objective)
  
    print("---SVM finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_VC,b_VC,c_VC=random_searchVC(train_features, train_labels,n_jobs,objective)
  
    print("---VotingClassification finished %s seconds ---" % round(time.time() - start_time,2))
    duration_time = time.time()
  
    a_AB,b_AB,c_AB=random_searchAB(train_features, train_labels,n_jobs,objective)
  
    print("---AdaBoost finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_NC,b_NC,c_NC=random_searchNC(train_features, train_labels,n_jobs,objective)
  
    print("---NeuralNetwork finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
  
  
    RF_ML = a_RF
    RF_ML.fit(train_features,train_labels)
    RF_ML.score(test_features,test_labels)
    RF_y_pred = RF_ML.predict(test_features)
  
  
    print("---RF fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    GB_ML = a_GB
    GB_ML.fit(train_features,train_labels)
    GB_ML.score(test_features,test_labels)
    GB_y_pred = GB_ML.predict(test_features)
  
  
    print("---GB fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    SVM_ML = a_SVM
    SVM_ML.fit(train_features,train_labels)
    SVM_ML.score(test_features,test_labels)
    SVM_y_pred = SVM_ML.predict(test_features)
  
  
    print("---SVM fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
  
    VC_ML = a_VC
    VC_ML.fit(train_features,train_labels)
    VC_ML.score(test_features,test_labels)
    VC_y_pred = VC_ML.predict(test_features)
  
  
    print("---VotingClassification fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    AB_ML = a_AB
    AB_ML.fit(train_features,train_labels)
    AB_ML.score(test_features,test_labels)
    AB_y_pred = AB_ML.predict(test_features)
  
  
    print("---AdaBoosting fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    NC_ML = a_NC
    NC_ML.fit(train_features,train_labels)
    NC_ML.score(test_features,test_labels)
    NC_y_pred = NC_ML.predict(test_features)
  
  
    print("---Neural Network fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    if objective == "f1":
        RF_score=f1_score(test_labels, RF_y_pred)
        GB_score=f1_score(test_labels, GB_y_pred)
        SVM_score=f1_score(test_labels, SVM_y_pred)
        VC_score=f1_score(test_labels, VC_y_pred)
        NC_score=f1_score(test_labels, NC_y_pred)
        AB_score=f1_score(test_labels, AB_y_pred)
    elif objective == "recall":
        RF_score=recall_score(test_labels, RF_y_pred)
        GB_score=recall_score(test_labels, GB_y_pred)
        SVM_score=recall_score(test_labels, SVM_y_pred)
        VC_score=recall_score(test_labels, VC_y_pred)
        NC_score=recall_score(test_labels, NC_y_pred)
        AB_score=recall_score(test_labels, AB_y_pred)
    elif objective == "accuracy":
        RF_score=accuracy_score(test_labels, RF_y_pred)
        GB_score=accuracy_score(test_labels, GB_y_pred)
        SVM_score=accuracy_score(test_labels, SVM_y_pred)
        VC_score=accuracy_score(test_labels, VC_y_pred)
        NC_score=accuracy_score(test_labels, NC_y_pred)
        AB_score=accuracy_score(test_labels, AB_y_pred)
    elif objective == "roc_auc":
        RF_score=roc_auc_score(test_labels, RF_y_pred)
        GB_score=roc_auc_score(test_labels, GB_y_pred)
        SVM_score=roc_auc_score(test_labels, SVM_y_pred)
        VC_score=roc_auc_score(test_labels, VC_y_pred)
        NC_score=roc_auc_score(test_labels, NC_y_pred)
        AB_score=roc_auc_score(test_labels, AB_y_pred)
    print ('RF:'+str(RF_score)+', GB:'+str(GB_score)+', SVM:'+str(SVM_score)+', VC:'+str(VC_score)+', AB:'+str(AB_score)+', NC:'+str(NC_score))
    score_list = [RF_score,GB_score,SVM_score,VC_score,NC_score,AB_score]
    if RF_score ==max(score_list):
        return a_RF,b_RF,c_RF
    elif GB_score ==max(score_list):
        return a_GB,b_GB,c_GB
    elif SVM_score ==max(score_list):
        return a_SVM,b_SVM,c_SVM
    elif VC_score ==max(score_list):
        return a_VC,b_VC,c_VC
    elif NC_score ==max(score_list):
        return a_NC,b_NC,c_NC
    elif AB_score ==max(score_list):
        return a_AB,b_AB,c_AB
    else:
        print('The error happens')


        

In [34]:
y_train, y_test, X_train, X_test = train_test_split( train_labels,train_features,
                                                    test_size=0.5, 
                                                    random_state=0)

In [35]:
a,b,c=best_estimater(X_train, y_train,X_test,y_test,'accuracy')

/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3

---RF finished 15.3 seconds ---
---GB finished 0.51 seconds ---
---SVM finished 0.14 seconds ---


/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_split.py

---VotingClassification finished 19.72 seconds ---
---AdaBoost finished 0.32 seconds ---


/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/dev-cory/.local/share/virtualenvs/kaggl

---NeuralNetwork finished 8.94 seconds ---
---RF fitting finished 0.03 seconds ---
---GB fitting finished 0.1 seconds ---
---SVM fitting finished 0.04 seconds ---
---VotingClassification fitting finished 0.13 seconds ---
---AdaBoosting fitting finished 0.09 seconds ---
---Neural Network fitting finished 0.64 seconds ---
RF:0.7668161434977578, GB:0.8183856502242153, SVM:0.6704035874439462, VC:0.7017937219730942, AB:0.7690582959641256, NC:0.7645739910313901


/home/dev-cory/.local/share/virtualenvs/kaggle-A36EqEnQ/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [36]:
a_txt = str(a)
print(a_txt)
print(b)
print(c)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.7887640449438202
{'learning_rate': 0.05}


In [42]:
import pickle
# write
with open('model/best_ml'+today_str+'.p', 'wb') as f:
    pickle.dump(a, f)

with open('model/best_ml_text'+today_str+'.p', 'wb') as f:
    pickle.dump(a_txt, f)    

In [43]:
ML = a
test_s2 = 0.5
y_train, y_test, X_train, X_test = train_test_split(train_labels,train_features,
                                                    test_size=test_s2,       
                                                    random_state=1)

ML.fit(X_train, y_train)
ML.score(X_test,y_test)
y_pred = ML.predict(X_test)
print("accuracy="+str(accuracy_score(y_test, y_pred)))
#print(average_precision_score(y_test,predict_y,average='weighted'))
print("f1="+str(f1_score(y_test, y_pred)))
print("precision="+str(precision_score(y_test, y_pred)))
print("recall="+str(recall_score(y_test, y_pred)))
#Vertucal:actual/ Horizontal:prediction
confusion_matrix(y_test,y_pred)

accuracy=0.773542600896861
f1=0.6793650793650793
precision=0.8106060606060606
recall=0.5846994535519126


array([[238,  25],
       [ 76, 107]])

In [60]:
with open('model/best_ml'+today_str+'.p', mode='rb') as f:
    ML2=pickle.load(f)
    



'''ML2=VotingClassifier(estimators=[('lr', LogisticRegression(C=199, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=1,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)), ('rf', Rand...=False, random_state=1,
            verbose=0, warm_start=False)), ('gnb', GaussianNB(priors=None))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)'''
ML2.fit(train_features,train_labels)

df_test=pd.read_csv('./data/test.csv')
df_label_test=transform_data(df_test)
df_label_test=df_label_test.drop(['PassengerId','Name','Ticket','Cabin',9],axis=1)

print(df_label_test.columns)
print(train2.columns)
y_pred=ML2.predict(df_label_test)

Index([   'Age',   'Fare',        1,        2,        3,      'C',      'Q',
            'S', 'female',   'male',        0,        1,        2,        3,
              4,        5,        8,        0,        1,        2,        3,
              4,        5,        6],
      dtype='object')
Index(['Survived',      'Age',     'Fare',          1,          2,          3,
              'C',        'Q',        'S',   'female',     'male',          0,
                1,          2,          3,          4,          5,          8,
                0,          1,          2,          3,          4,          5,
                6],
      dtype='object')


In [18]:
y_pred2=pd.DataFrame(y_pred)
df_answer=pd.concat([df_test["ID_code"],y_pred2],axis=1)
df_answer.to_csv('./santandar/submission.csv',index=False)